In [1]:
import sys
sys.path.append('../')
import latexStrings as ls
import numpy as np
import matplotlib.pyplot as plt
from math import *
from IPython.display import Latex
import odesolver
from scipy import optimize as op
import scipy.linalg as linear

# Shooting Method

Tomemos el siguiente PVI: 
\begin{equation} 
\label{eq:3}
 \begin{array}{c} y'' = 10 y' (1 - y)\end{array} 
\qquad \mbox{con} \qquad \begin{array}{c}
y(0)=1 \\ y(1) = 1 - pi/20 \end{array} \qquad \mbox{para } t\in [0,1] \qquad 

\mbox{Que podemos reescribir como un sistema de ecuaciones de primer orden de la siguiente manera:} \\
\begin{array}{c} y_1' = y_2 \\
y_2' = 10 y_2 (1 - y_1) \end{array}

\end{equation}

con solución exacta
 está dada por :
\begin{equation}
 \begin{array}{c} y(t) = 1-\frac{\pi}{20} \tan(\frac{\pi}{4}t) \end{array}
\end{equation}


así, definimos f como

In [2]:
f = lambda t, y : np.array([y[1], 10*y[1]*(1-y[0])])

In [3]:
exact = lambda t : 1-pi/20*tan(pi/4*t)

Para aplicar Shooting Method, tomamos una aproximación inicial de y' = 1, o bien,

$\overrightarrow{y}(t)=\left(\begin{array}{c} y_1(t) \\
y_2(t)\end{array} \right) = \left(\begin{array}{c} 1 \\ 1 \right) \end{array}

y utilizaremos el método de Runge Kutta 4 con 1000 pasos para aproximar la función en y(1)

In [4]:
y0 = [1, 0]

In [5]:
T, W = odesolver.solve(f, y0, (0,1), 1000,  method = 'rk4')

In [6]:
ans = W[0][-1]
ans

1.0

Veamos que tan acertada fue nuestra elección

In [7]:
exact(1) == 1-pi/20

True

In [8]:
abs(ans - (1 - pi/20))

0.15707963267948966

Parece ser que nuestra aproximación inicial sobreestimó la solución. Usaremos el método de Newton para seguir generando aproximaciones.

Definimos una función 
$F : \mathbb{R} \rightarrow \mathbb{R}$
que toma una aproximación inical y regresa la distancia 

In [9]:
F = lambda yp0 : odesolver.solve(f, [y0, yp0], I, 1000,  method = 'rk4')[1][0][-1]

In [10]:
I = (0,1)

In [11]:
y0 = 1

In [12]:
F(1)

1.4371120401610173

In [13]:
# op.newton no puede outputear el número de iteraciones, pero a prueba y error encontramos que toma 7 para converger

In [14]:
y_init = odesolver.shooting(f, 1, 0, exact(1), (0,1), tol = 1e-10, maxiter=7)
#y_init = shooting(f, 1, 0, 1 - pi/20, (0,1), tol = 1e-10)
y_init

-0.12337005501362536

Tomando y_init como aproximación inicial a y'

In [15]:
y0 = [1, y_init]

In [16]:
_, W = odesolver.solve(f, y0, (0,1), 1000,  method = 'rk4')
ans = W[0][-1]

In [17]:
abs(ans - (1 - pi/20))

9.992007221626409e-16

Observamos que con esta aprximación inicial se satisface el BVP